# FinBERT Example Notebook

This notebooks shows how to train and use the FinBERT pre-trained language model for financial sentiment analysis.

## Modules 

In [33]:
from pathlib import Path
import sys
sys.path.append('..')
import argparse
import shutil
import os
import logging
from textblob import TextBlob

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import *

from finbert.finbert import *
import finbert.utils as tools
from pprint import pprint
from sklearn.metrics import classification_report
%load_ext autoreload
%autoreload 2

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

## Prepare the model

### Setting path variables:
1. `lm_path`: the path for the pre-trained language model (If vanilla Bert is used then no need to set this one).
2. `cl_path`: the path where the classification model is saved.
3. `cl_data_path`: the path of the directory that contains the data files of `train.csv`, `validation.csv`, `test.csv`.
---

In the initialization of `bertmodel`, we can either use the original pre-trained weights from Google by giving `bm = 'bert-base-uncased`, or our further pre-trained language model by `bm = lm_path`


---
All of the configurations with the model is controlled with the `config` variable. 

In [35]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
cl_data_path = project_dir/'data'/'sentiment_data'

###  Configuring training parameters

You can find the explanations of the training parameters in the class docsctrings. 

In [36]:
# Clean the cl_path
try:
    shutil.rmtree(cl_path) 
except:
    pass

bertmodel = BertForSequenceClassification.from_pretrained(lm_path,cache_dir=None, num_labels=3)


config = Config(   data_dir=cl_data_path,
                   bert_model=bertmodel,
                   num_train_epochs=4,
                   model_dir=cl_path,
                   max_seq_length = 48,
                   train_batch_size = 32,
                   learning_rate = 2e-5,
                   output_mode='classification',
                   warm_up_proportion=0.2,
                   local_rank=-1,
                   discriminate=True,
                   gradual_unfreeze=True )

03/23/2020 15:55:09 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /home/ubuntu/public_final_test/github/models/language_model/finbertTRC2 from cache at /home/ubuntu/public_final_test/github/models/language_model/finbertTRC2
03/23/2020 15:55:09 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

03/23/2020 15:55:12 - INFO - pytorch_pretrained_bert.modeling -   Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
03/23/2020 15:55:12 - INFO - pytorch_pretrained_bert.modeling -   Weights from pretrained model not used in BertForSequenceClassification: ['cls.predictions.bias', 'cls.predic

`finbert` is our main class that encapsulates all the functionality. The list of class labels should be given in the prepare_model method call with label_list parameter.

In [37]:
finbert = FinBert(config)

In [38]:
finbert.prepare_model(label_list=['positive','negative','neutral'])

03/23/2020 15:55:17 - INFO - finbert.finbert -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
03/23/2020 15:55:17 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Fine-tune the model

In [39]:
# Get the training examples
train_data = finbert.get_data('train')

In [40]:
model = finbert.create_the_model()

### [Optional] Fine-tune only a subset of the model
The variable `freeze` determines the last layer (out of 12) to be freezed. You can skip this part if you want to fine-tune the whole model.

<span style="color:red">Important: </span>
Execute this step if you want a shorter training time in the expense of accuracy.

In [ ]:
# This is for fine-tuning a subset of the model.

freeze = 11

for param in model.bert.embeddings.parameters():
    param.requires_grad = False
    
for i in range(freeze):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = False

### Training

In [41]:
trained_model = finbert.train(train_examples = train_data, model = model)

03/23/2020 15:55:27 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:55:27 - INFO - finbert.utils -   guid: train-1
03/23/2020 15:55:27 - INFO - finbert.utils -   tokens: [CLS] weber convinced his friend ray os ##trom , who owned a lake street sporting goods store , to sell the lure ##s . [SEP]
03/23/2020 15:55:27 - INFO - finbert.utils -   input_ids: 101 13351 6427 2010 2767 4097 9808 13887 1010 2040 3079 1037 2697 2395 7419 5350 3573 1010 2000 5271 1996 17256 2015 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:27 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:27 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:27 - INFO - finbert.utils -   label: neutral (id = 2)
03/23/2020 15:55:28 - INFO - finbert.finbert -   ***** Loading data *****
03/23/2020

03/23/2020 15:55:50 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:55:50 - INFO - finbert.utils -   guid: validation-1
03/23/2020 15:55:50 - INFO - finbert.utils -   tokens: [CLS] the fair value of the company ' s investment properties grew to eu ##r 2 . 80 ##3 billion at the end of march 2009 from eu ##r 2 . 69 ##1 million a year ago . [SEP]
03/23/2020 15:55:50 - INFO - finbert.utils -   input_ids: 101 1996 4189 3643 1997 1996 2194 1005 1055 5211 5144 3473 2000 7327 2099 1016 1012 3770 2509 4551 2012 1996 2203 1997 2233 2268 2013 7327 2099 1016 1012 6353 2487 2454 1037 2095 3283 1012 102 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:50 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:50 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:55:50 - INFO - finbert.utils -   label: positive (i

03/23/2020 15:55:51 - INFO - finbert.finbert -   ***** Loading data *****
03/23/2020 15:55:51 - INFO - finbert.finbert -     Num examples = 776
03/23/2020 15:55:51 - INFO - finbert.finbert -     Batch size = 32
03/23/2020 15:55:51 - INFO - finbert.finbert -     Num steps = 96



Validation losses: [0.94504958152771]
No best model found


Epoch:  25%|██▌       | 1/4 [00:27<01:23, 27.70s/it]

...

Epoch:  75%|███████▌  | 3/4 [02:03<00:38, 38.71s/it]

03/23/2020 15:58:28 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
03/23/2020 15:58:29 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
03/23/2020 15:58:29 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
03/23/2020 15:58:29 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:58:29 - INFO - finbert.utils -   guid: validation-1
03/23/2020 15:58:29 - INFO - finbert.utils -   tokens: [CLS] the fair value of the company ' s investment properties grew to eu ##r 2 . 80 ##3 billion at the end of march 2009 from eu ##r 2 . 69 ##1 million a year ago . [SEP]
03/23/2020 15:58:29 - INFO - finbert

03/23/2020 15:58:30 - INFO - finbert.finbert -   ***** Loading data *****
03/23/2020 15:58:30 - INFO - finbert.finbert -     Num examples = 776
03/23/2020 15:58:30 - INFO - finbert.finbert -     Batch size = 32
03/23/2020 15:58:30 - INFO - finbert.finbert -     Num steps = 96


Epoch: 100%|██████████| 4/4 [03:06<00:00, 46.55s/it]


Validation losses: [0.94504958152771, 0.40575565814971926, 0.34499375462532045, 0.34853786170482637]


## Test the model

`bert.evaluate` outputs the DataFrame, where true labels and logit values for each example is given

In [42]:
test_data = finbert.get_data('test')

In [43]:
results = finbert.evaluate(examples=test_data, model=trained_model)

03/23/2020 15:58:41 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:58:41 - INFO - finbert.utils -   guid: test-1
03/23/2020 15:58:41 - INFO - finbert.utils -   tokens: [CLS] alt ##imo and tel ##ias ##one ##ra said yesterday that us ##man ##ov would be welcome to join the new company . [SEP]
03/23/2020 15:58:41 - INFO - finbert.utils -   input_ids: 101 12456 16339 1998 10093 7951 5643 2527 2056 7483 2008 2149 2386 4492 2052 2022 6160 2000 3693 1996 2047 2194 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:58:41 - INFO - finbert.utils -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:58:41 - INFO - finbert.utils -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/23/2020 15:58:41 - INFO - finbert.utils -   label: positive (id = 0)
03/23/2020 15:58:42 - INFO - finbert.finbert -   ***** Loading data *****
03/23/2020 15:

### Prepare the classification report

In [44]:
def report(df, cols=['label','prediction','logits']):
    #print('Validation loss:{0:.2f}'.format(metrics['best_validation_loss']))
    cs = CrossEntropyLoss(weight=finbert.class_weights)
    loss = cs(torch.tensor(list(df[cols[2]])),torch.tensor(list(df[cols[0]])))
    print("Loss:{0:.2f}".format(loss))
    print("Accuracy:{0:.2f}".format((df[cols[0]] == df[cols[1]]).sum() / df.shape[0]) )
    print("\nClassification Report:")
    print(classification_report(df[cols[0]], df[cols[1]]))

In [45]:
results['prediction'] = results.predictions.apply(lambda x: np.argmax(x,axis=0))

In [46]:
report(results,cols=['labels','prediction','predictions'])

Loss:0.36
Accuracy:0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.83      0.78       238
           1       0.78      0.94      0.85       125
           2       0.93      0.84      0.88       606

    accuracy                           0.85       969
   macro avg       0.81      0.87      0.84       969
weighted avg       0.86      0.85      0.85       969



### Get predictions

With the `predict` function, given a piece of text, we split it into a list of sentences and then predict sentiment for each sentence. The output is written into a dataframe. Predictions are represented in three different columns: 

1) `logit`: probabilities for each class

2) `prediction`: predicted label

3) `sentiment_score`: sentiment score calculated as: probability of positive - probability of negative

Below we analyze a paragraph taken out of [this](https://www.economist.com/finance-and-economics/2019/01/03/a-profit-warning-from-apple-jolts-markets) article from The Economist. For comparison purposes, we also put the sentiments predicted with TextBlob.
> Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened. The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. Yields on government bonds fell as investors fled to the traditional haven in a market storm.

In [47]:
text = "Later that day Apple said it was revising down its earnings expectations in \
the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China. \
The news rapidly infected financial markets. Apple’s share price fell by around 7% in after-hours \
trading and the decline was extended to more than 10% when the market opened. The dollar fell \
by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering \
some ground. Asian stockmarkets closed down on January 3rd and European ones opened lower. \
Yields on government bonds fell as investors fled to the traditional haven in a market storm."

In [48]:
cl_path = project_dir/'models'/'classifier_model'/'finbert-sentiment'
model = BertForSequenceClassification.from_pretrained(cl_path, cache_dir=None, num_labels=3)

03/23/2020 15:59:13 - INFO - pytorch_pretrained_bert.modeling -   loading archive file /home/ubuntu/public_final_test/github/models/classifier_model/finbert-sentiment from cache at /home/ubuntu/public_final_test/github/models/classifier_model/finbert-sentiment
03/23/2020 15:59:13 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [49]:
result = predict(text,model)

03/23/2020 15:59:18 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/23/2020 15:59:18 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:59:18 - INFO - finbert.utils -   guid: 0
03/23/2020 15:59:18 - INFO - finbert.utils -   tokens: [CLS] later that day apple said it was rev ##ising down its earnings expectations in the fourth quarter of 2018 , largely because of lower sales and signs of economic weakness in china . [SEP]
03/23/2020 15:59:18 - INFO - finbert.utils -   input_ids: 101 2101 2008 2154 6207 2056 2009 2001 7065 9355 2091 2049 16565 10908 1999 1996 2959 4284 1997 2760 1010 4321 2138 1997 2896 4341 1998 5751 1997 3171 11251 1999 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [50]:
blob = TextBlob(text)
result['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]
result

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,"Later that day Apple said it was revising down its earnings expectations in the fourth quarter of 2018, largely because of lower sales and signs of economic weakness in China.","[0.019688552, 0.943051, 0.03726054]",negative,-0.923362,0.051746
1,The news rapidly infected financial markets.,"[0.03636261, 0.8453178, 0.11831961]",negative,-0.808955,0.000000
2,Apple’s share price fell by around 7% in after-hours trading and the decline was extended to more than 10% when the market opened.,"[0.011015116, 0.97663385, 0.012351025]",negative,-0.965619,0.500000
3,"The dollar fell by 3.7% against the yen in a matter of minutes after the announcement, before rapidly recovering some ground.","[0.041714344, 0.94278675, 0.0154988365]",negative,-0.901072,0.000000
4,Asian stockmarkets closed down on January 3rd and European ones opened lower.,"[0.00964421, 0.9713175, 0.019038342]",negative,-0.961673,-0.051111
5,Yields on government bonds fell as investors fled to the traditional haven in a market storm.,"[0.024553798, 0.93890834, 0.03653784]",negative,-0.914355,0.000000


In [51]:
print(f'Average sentiment is %.2f.' % (result.sentiment_score.mean()))

Average sentiment is -0.91.


Here is another example

In [52]:
text2 = "Shares in the spin-off of South African e-commerce group Naspers surged more than 25% \
in the first minutes of their market debut in Amsterdam on Wednesday. Bob van Dijk, CEO of \
Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the \
Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019. REUTERS/Piroschka van de Wouw \
Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a \
31% stake in Chinese tech titan Tencent. There is 'way more demand than is even available, so that’s \
good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg. 'It’s going to be an interesting \
hour of trade after opening this morning.' Euronext had given an indicative price of 58.70 euros \
per share for Prosus, implying a market value of 95.3 billion euros ($105 billion). The shares \
jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT."

In [53]:
result2 = predict(text2,model)
blob = TextBlob(text2)
result2['textblob_prediction'] = [sentence.sentiment.polarity for sentence in blob.sentences]

03/23/2020 15:59:47 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
03/23/2020 15:59:47 - INFO - finbert.utils -   *** Example ***
03/23/2020 15:59:47 - INFO - finbert.utils -   guid: 0
03/23/2020 15:59:47 - INFO - finbert.utils -   tokens: [CLS] shares in the spin - off of south african e - commerce group nas ##pers surged more than 25 % in the first minutes of their market debut in amsterdam on wednesday . [SEP]
03/23/2020 15:59:47 - INFO - finbert.utils -   input_ids: 101 6661 1999 1996 6714 1011 2125 1997 2148 3060 1041 1011 6236 2177 17235 7347 18852 2062 2084 2423 1003 1999 1996 2034 2781 1997 2037 3006 2834 1999 7598 2006 9317 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
03/

In [54]:
result2

,sentence,logit,prediction,sentiment_score,textblob_prediction
0,Shares in the spin-off of South African e-commerce group Naspers surged more than 25% in the first minutes of their market debut in Amsterdam on Wednesday.,"[0.94092953, 0.020505333, 0.038565084]",positive,0.920424,0.250000
1,"Bob van Dijk, CEO of Naspers and Prosus Group poses at Amsterdam's stock exchange, as Prosus begins trading on the Euronext stock exchange in Amsterdam, Netherlands, September 11, 2019.","[0.038308024, 0.009095702, 0.9525962]",neutral,0.029212,0.000000
2,"REUTERS/Piroschka van de Wouw Prosus comprises Naspers’ global empire of consumer internet assets, with the jewel in the crown a 31% stake in Chinese tech titan Tencent.","[0.081532344, 0.004831061, 0.91363657]",neutral,0.076701,0.000000
3,"There is 'way more demand than is even available, so that’s good,' said the CEO of Euronext Amsterdam, Maurice van Tilburg.","[0.87810785, 0.012375466, 0.10951674]",positive,0.865732,0.533333
4,'It’s going to be an interesting hour of trade after opening this morning.',"[0.3788318, 0.020865666, 0.6003026]",neutral,0.357966,0.500000
5,"Euronext had given an indicative price of 58.70 euros per share for Prosus, implying a market value of 95.3 billion euros ($105 billion).","[0.13808341, 0.012395262, 0.84952134]",neutral,0.125688,0.000000
6,The shares jumped to 76 euros on opening and were trading at 75 euros at 0719 GMT.,"[0.7504708, 0.018825786, 0.23070341]",positive,0.731645,0.000000


In [55]:
print(f'Average sentiment is %.2f.' % (result2.sentiment_score.mean()))

Average sentiment is 0.44.
